<a href="https://colab.research.google.com/github/mnijhuis-dnb/OpenSourceWorkshop-main/blob/master/Explainable_AI/analyse_data_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1EE5KO2VXEbxwZ2P0k-PP03NsNRL4PBPJ
!gdown 1atPsRYwb-mWq585wlz8iGtw5uaKck4N8

In [ ]:
!pip install eli5
!pip install shap
!pip install pdpbox

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv('/content/mortage_data.csv', index_col=0)

In [ ]:
float_columns = ['Original Interest Rate', 'Original UPB', 
    'Origination Date', 'Remaining Months To Maturity Original', 
    'Original Loan to Value Ratio (LTV)', 'Original Combined Loan to Value Ratio (CLTV)', 'Number of Borrowers', 
    'Debt-To-Income (DTI)', 'Borrower Credit Score at Origination', 'Co-Borrower Credit Score at Origination', 
    'Zip Code Short', 'Total Principal Current Original', 'Borrower Age']

In [ ]:
one_hot_columns = ['Channel', 'First Time Home Buyer Indicator', 'Loan Purpose', 
    'Property Type', 'Occupancy Status', 'Property State', 'Prepayment Penalty Indicator', 
    'Borrower Education Level', 'Borrower Employment', 'Relocation Mortgage Indicator', 'High Balance Loan Indicator']

In [ ]:
keep_columns = ['Channel', 'Original Interest Rate', 
    'Original UPB', 'Origination Date', 
    'Remaining Months To Maturity Original', 'Original Loan to Value Ratio (LTV)', 
    'Original Combined Loan to Value Ratio (CLTV)', 'Number of Borrowers', 
    'Debt-To-Income (DTI)', 'Borrower Credit Score at Origination', 'Co-Borrower Credit Score at Origination', 
    'First Time Home Buyer Indicator', 'Loan Purpose', 'Property Type', 'Occupancy Status', 'Property State', 
    'Zip Code Short', 'Prepayment Penalty Indicator', 
    'Total Principal Current Original', 'Relocation Mortgage Indicator', 
    'High Balance Loan Indicator', 'Borrower Age', 'Borrower Education Level', 
    'Borrower Employment']

In [ ]:
df.loc[df['Prepayment Penalty Indicator'].isna(),'Prepayment Penalty Indicator'] = 'nan'
df.loc[df['Interest Only Loan Indicator Original'].isna(),'Interest Only Loan Indicator Original'] = 'nan'
df.loc[df['Alternative Delinquency Resolution Original']==7.0,'Alternative Delinquency Resolution Original'] = '7'


In [ ]:
df_x = df[keep_columns].copy()
df_y = (df['Total Writeoff'].fillna(0)>0).astype(int)

In [ ]:
for col in float_columns:
    df_x[col] = pd.to_numeric(df_x[col],errors='coerce',downcast='integer').fillna(-1)

In [ ]:
for col in one_hot_columns:
    df_x = pd.concat([df_x.drop(columns=[col]), pd.get_dummies(df_x[col], prefix=col + ' ')],axis=1)

In [ ]:
def downsample(data_x, data_y, num):

    ind = np.r_[:len(data_y)][data_y>0]
    idx = np.r_[:len(data_y)][data_y<=0]
    indexes_a = np.random.choice(ind, num)
    indexes_b = np.random.choice(idx, num)
    data_x = np.vstack((data_x[indexes_a], data_x[indexes_b])) 
    data_y = np.hstack((data_y[indexes_a], data_y[indexes_b]))

    return data_x, data_y


def sample(data_x, data_y, num):

    indexes = np.zeros(len(data_x), dtype=bool)
    indexes[np.random.choice(np.r_[:len(data_x)], num)] = True
    train_x, train_y = downsample(data_x[indexes], data_y[indexes], num)
    test_x = data_x[np.bitwise_not(indexes)]
    test_y = data_y[np.bitwise_not(indexes)]

    return train_x, train_y, test_x, test_y

In [ ]:
train_x, train_y, test_x, test_y = sample(df_x.values, df_y.values, 300000)
rfc = RandomForestClassifier(n_estimators=50, min_samples_split=100, max_depth=10, bootstrap=True, max_samples=0.05)

In [ ]:
rfc.fit(train_x, train_y)

In [ ]:
rfc.predict(test_x)